# Retrieval Augmented Generation

In [ ]:
!pip install -q langchain langchain_aws langchain_openai dotenv langchain-ollama

In [29]:
from langchain_aws import ChatBedrock
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()

llm = ChatOpenAI(
    model="meta-llama/Llama-3.1-8B-Instruct",  
    openai_api_base=f"http://{os.getenv('LLM_IP')}:8000/v1",
    openai_api_key="EMPTY"  # vLLM doesn’t require real keys
)


In [30]:
ai_msg = llm.invoke("Which VLANs are reverved in my network?")
print(ai_msg.content)

I'm not able to access your network or its configuration. However, I can provide some general information about VLAN reservation.

VLAN (Virtual Local Area Network) reservations can vary depending on the network and its purpose. Here are some common VLAN reservations:

1. **VLAN 1**: This is often reserved for the management VLAN, which is used for network management and administration.
2. **VLAN 1002-1005**: These VLANs are reserved by the IEEE (Institute of Electrical and Electronics Engineers) for special purposes:
	* VLAN 1002: Reserved for the Cisco proprietary protocol (not a standard VLAN).
	* VLAN 1003: Reserved for the IEEE 802.1Q VLAN management.
	* VLAN 1004: Reserved for the IEEE 802.1Q VLAN management (same as VLAN 1003).
	* VLAN 1005: Reserved for the IEEE 802.1Q VLAN management (same as VLAN 1003 and VLAN 1004).
3. **VLAN 4094**: This VLAN is reserved by the IEEE as a "default" VLAN, which means it's not used for any specific purpose and can be used as a fallback VLAN.



## Ingestion Pipeline (Add data to Vector DB)
### Load

In [31]:
from langchain.document_loaders import TextLoader

loader = TextLoader("acme_policies.md")
documents = loader.load()


### Split

In [32]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

In [34]:
md_splits = md_splitter.split_text(documents[0].page_content)
md_splits[5].metadata

{'Header 1': 'Policies', 'Header 2': 'Quality of Service (QoS)'}

In [35]:
md_splits[5].page_content

'1. All edge interfaces must implement QoS policies to prioritize voice and critical applications.  \n```\npolicy-map EDGE_POLICY\nclass VOICE\npriority 500\nclass CRITICAL_APP\nbandwidth percent 30\n```  \n2. Default traffic should be marked as Best Effort (BE).'

In [33]:
len(md_splits)

9

### Embedd

In [36]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_aws import BedrockEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from dotenv import load_dotenv
import os
load_dotenv()

embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    base_url=f'http://{os.getenv("EMBEDDING_IP")}:8081'
)

vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(documents=md_splits)

['440bc5a3-8369-43f0-962b-973b9e6abf34',
 '0c52ef6d-f478-4fca-a72d-53791b57511b',
 'af591d8d-c697-448f-9f83-835a3cfa89e3',
 '3f33cfe1-b416-4393-a9ed-b6a70ff815bd',
 'bdf9e939-355d-49dc-ab4f-da04082cb99b',
 '5e271ae3-0f8d-4bf5-9c88-c802ce2a316b',
 '975bc5e7-332e-4b7b-894c-40418602aa0f',
 'f146133a-0559-4494-9d2a-9797cb2d7e80',
 'ca086c63-d4c0-492c-b206-45c8e8fd64b0']

In [37]:
for index, (id, doc) in enumerate(vector_store.store.items()):
    if index < 2:
        print(doc['text'])
        print(f"{id}: {doc['vector'][0:5]} ({len(doc['vector'])})")
        print('----------------------------------------------------------------------------------------------------')

The following document states the configuration policies of the devices of the ACME company.
440bc5a3-8369-43f0-962b-973b9e6abf34: [0.009143354, 0.064187326, -0.17247662, -0.024310747, 0.053223427] (768)
----------------------------------------------------------------------------------------------------
1. Routable addresses used on physical Interfaces (E.g: Ethernet 1/2), Port-Chanels (port-channel1) or SVI Interfaces must be taken from the CIDR 10.1.0.0/16  
Example:
```
interface vlan 123
no switchport
ip address 10.1.1.254/24
```  
2. IP Addresses assigned to Router IDs (RID) of routing protocols such as OSPF, EIGRP or BGP must be taken from the CIDR 172.20.0.0/16  
Example:
```
router ospf OSPF1
router-id  172.20.1.5
```
0c52ef6d-f478-4fca-a72d-53791b57511b: [-0.012690579, 0.043827474, -0.15462454, -0.030858142, 0.07985148] (768)
----------------------------------------------------------------------------------------------------


### Similarity Search

In [38]:
query = "approved security mechanisms"
results = vector_store.similarity_search(query, k=2)
for result in results:
    print(result.page_content)

1. The only valid hashing protocol for payload integrity is MD5. This applies for SNMP configurations and authenticated BGP peerings.  
```
snmp-server user MYUSER MYGROUP v3 auth sha <password> priv aes 256 <password>
```  
2. The only valid authentication protocol for payload encryption is AES 256.
The following document states the configuration policies of the devices of the ACME company.


### RAG Pipeline (Retrieve context from Vector DB)

In [ ]:
from langchain import hub
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate

retrieval_qa_chat_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a network engineer assistant. Use the following documentation to answer the question.'
If the answer is not in the documentation, say you don’t know.

Documentation:
{context}

Question: {question}

Answer:
"""
)

In [40]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    
rag_chain = (
    {'context': vector_store.as_retriever() | format_docs, 'question' : RunnablePassthrough()} |
    retrieval_qa_chat_prompt |
    llm
)

result = rag_chain.invoke("Which IP address can I use for router ID?")
print(result.content)

You can use an IP address from the CIDR 172.20.0.0/16 for the router ID of routing protocols such as OSPF, EIGRP, or BGP.
